# 单条调用（采用免费模型THUDM/GLM-Z1-9B-0414）

In [ ]:
import requests

url = "https://api.siliconflow.cn/v1/chat/completions"

payload = {
    "model": "THUDM/GLM-Z1-9B-0414",
    "messages": [
        {
            "role": "user",
            "content": "你好"
        }
    ]
}
headers = {
    "Authorization": "Bearer 你的API Key",
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

print(response.json()['choices'][0]['message']['content'])


你好！有什么可以帮你的吗？无论是问题解答、信息查询，还是闲聊吐槽，我都在这儿呢～ 😄


# 单次调用完成情感分类

In [ ]:
import requests
import time

url = "https://api.siliconflow.cn/v1/chat/completions"
headers = {
    "Authorization": "Bearer 你的API Key",
    "Content-Type": "application/json"
}

payload = {
    "model": "THUDM/GLM-Z1-9B-0414",
    "messages": [
        {
            "role": "user",
            "content": """你是一个情感分类专家，善于判断文本的情感倾向。现在，我需要你协助我完成文本情感分类任务，以下是任务描述以及具体的要求：
            1. 我会输入一段酒店评论，你需要协助我判断这段酒店评论的情感倾向。
            2. 情感倾向分为积极情感与消极情感。
            3. 如果你认为这段评论属于积极情感，则返回1；如果属于消极情感，则返回0.
            4. 不要输出任何其他内容，只输出0或1.

            input：环境交通都不错，房间也可以，下次还会再来
            output：
            """
        }
    ]
}

def get_sentiment_result(max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            response = requests.post(url, json=payload, headers=headers)
            response.raise_for_status()
            content = response.json()['choices'][0]['message']['content'].strip()

            # 清洗结果，仅保留0或1
            clean_output = content.strip().replace("\n", "").replace(" ", "")
            if clean_output in ["0", "1", 0, 1]:
                return clean_output
            else:
                print(f"⚠️ 无效输出：{content}，重新请求中...（第 {retries+1} 次）")
                retries += 1
                time.sleep(1)  # 避免频繁请求
        except Exception as e:
            print(f"❌ 请求或解析出错：{e}，重新尝试中...（第 {retries+1} 次）")
            retries += 1
            time.sleep(1)

    raise RuntimeError("多次请求仍未获得有效结果（0或1）")

# 调用函数获取结果
result = get_sentiment_result()
print(result)


1


# 从本地读取数据，完成批量情感分类

从本地读取数据，`review`字段是评论（即待情感分类的文本），`label`字段是已经打好的情感标签（1代表积极，0代表消极）。

In [2]:
import pandas as pd
df = pd.read_csv('data_sample.csv')

df.head()

,label,review
0,1,房间不大，但相当舒适．距离热闹的酒吧集中区也很近，方便．环境干净＼清净，适合睡大觉．服务中规...
1,1,酒店就是离市区太远。离新国际展会也不近。我的客人还算满意。
2,1,帮同事订过很多次，酒店位置很好，很方便。房间也不错。
3,0,"就值5元的浴盐瓶要赔40元,这不是主要的,最主要的是质量差的东西能伤人,我打开没用力就破,我..."
4,0,帮朋友订的，反映不是太好，想必也是和城市规模有关系，宜昌的四星肯定是不能和大城市的四星相比。...


批量完成情感分类任务

In [ ]:
import requests
import time
import pandas as pd
from tqdm import tqdm  
import json

# LLM API配置
url = "https://api.siliconflow.cn/v1/chat/completions"
headers = {
    "Authorization": "Bearer 你的API key",  # 替换为你的真实 API key
    "Content-Type": "application/json"
}

def get_sentiment_result(review_text, max_retries=5):
    """调用LLM判断单条评论情感"""
    base_prompt = f"""你是一个情感分类专家，善于判断文本的情感倾向。现在，我需要你协助我完成文本情感分类任务，以下是任务描述以及具体的要求：
1. 我会输入一段酒店评论，你需要协助我判断这段酒店评论的情感倾向。
2. 情感倾向分为积极情感与消极情感。
3. 如果你认为这段评论属于积极情感，则返回1；如果属于消极情感，则返回0.
4. 不要输出任何其他内容，只输出0或1.

input：{review_text}
output：
"""

    payload = {
        "model": "THUDM/GLM-Z1-9B-0414",
        "messages": [
            {"role": "user", "content": base_prompt}
        ]
    }

    retries = 0
    while retries < max_retries:
        try:
            payload_json = json.dumps(payload, ensure_ascii=False).encode('utf-8')
            response = requests.post(url, data=payload_json, headers=headers)
            response.raise_for_status()
            content = response.json()['choices'][0]['message']['content'].strip()
            clean_output = content.replace("\n", "").replace(" ", "").strip()
            if clean_output in ["0", "1"]:
                return int(clean_output)
            else:
                print(f"⚠️ 非法输出: {clean_output}，第 {retries + 1} 次重试...")
                retries += 1
                time.sleep(1)
        except Exception as e:
            print(f"❌ 异常：{e}，第 {retries + 1} 次重试...")
            retries += 1
            time.sleep(1)

    return None  # 如果重试失败，返回空值

# 批量情感分析 + tqdm进度条
results = []
for i, row in tqdm(df.iterrows(), total=len(df), desc="情感分析中"):
    review = row['review']
    sentiment = get_sentiment_result(review)
    results.append(sentiment)

# 添加结果到DataFrame
df['llm_output'] = results

# 保存结果
df.to_csv('data_singlethread.csv', index=False)
df.head()

情感分析中: 100%|██████████| 100/100 [09:01<00:00,  5.41s/it]


,label,review,llm_output
0,1,房间不大，但相当舒适．距离热闹的酒吧集中区也很近，方便．环境干净＼清净，适合睡大觉．服务中规...,1
1,1,酒店就是离市区太远。离新国际展会也不近。我的客人还算满意。,0
2,1,帮同事订过很多次，酒店位置很好，很方便。房间也不错。,1
3,0,"就值5元的浴盐瓶要赔40元,这不是主要的,最主要的是质量差的东西能伤人,我打开没用力就破,我...",0
4,0,帮朋友订的，反映不是太好，想必也是和城市规模有关系，宜昌的四星肯定是不能和大城市的四星相比。...,0


# 多线程调用大模型，提升效率

- 这里使用了`concurrent`库来实现多线程调用大模型，大大提升了调用的速率。如果有同学对多线程（来自计算机操作系统的知识点）这个概念感兴趣，可以课后去了解一下。
- 注意，多线程调用大模型的速率依旧是有上限的，因为厂商会限定模型的调用速率上限，这是个硬性限制，没法突破。

In [ ]:
import requests
import time
import pandas as pd
from tqdm import tqdm  
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

url = "https://api.siliconflow.cn/v1/chat/completions"

api_keys = [
    "sk-123",
    "sk-xxxxx2",
    "sk-xxxxx3",
    "sk-xxxxx4",
    "sk-xxxxx5"
] # 在Silicon Flow中生成5个api key，并填入列表api_keys中。例如，第一个api key为"sk-123"，如图所示依次替换后4个str即可。

def get_sentiment_result(review_text, api_key, max_retries=5):
    base_prompt = f"""你是一个情感分类专家，善于判断文本的情感倾向。现在，我需要你协助我完成文本情感分类任务，以下是任务描述以及具体的要求：
1. 我会输入一段酒店评论，你需要协助我判断这段酒店评论的情感倾向。
2. 情感倾向分为积极情感与消极情感。
3. 如果你认为这段评论属于积极情感，则返回1；如果属于消极情感，则返回0.
4. 不要输出任何其他内容，只输出0或1.

input：{review_text}
output：
"""

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "THUDM/GLM-Z1-9B-0414",
        "messages": [
            {"role": "user", "content": base_prompt}
        ]
    }

    retries = 0
    while retries < max_retries:
        try:
            payload_json = json.dumps(payload, ensure_ascii=False).encode('utf-8')
            response = requests.post(url, data=payload_json, headers=headers)
            response.raise_for_status()
            content = response.json()['choices'][0]['message']['content'].strip()
            clean_output = content.replace("\n", "").replace(" ", "").strip()
            if clean_output in ["0", "1"]:
                return int(clean_output)
            else:
                print(f"⚠️ 非法输出: {clean_output}，第 {retries + 1} 次重试...")
                retries += 1
                time.sleep(1)
        except Exception as e:
            print(f"❌ 异常：{e}，第 {retries + 1} 次重试...")
            retries += 1
            time.sleep(1)

    return None

# 线程池最大5线程
max_workers = 5

results = [None] * len(df)

def task(idx, review):
    api_key = api_keys[idx % len(api_keys)]  # 轮流选择api key
    sentiment = get_sentiment_result(review, api_key)
    return idx, sentiment

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(task, i, row['review']) for i, row in df.iterrows()]
    for future in tqdm(as_completed(futures), total=len(df), desc="情感分析中"):
        idx, sentiment = future.result()
        results[idx] = sentiment

df['llm_output'] = results
df.to_csv('data_multithread.csv', index=False)
df.head()


情感分析中:  40%|████      | 40/100 [00:45<00:50,  1.20it/s]

⚠️ 非法输出: 根据该月的借款数据，我们可以推测该用户的借款周期大约为：**15天**具体推演过程：1.假设每月借款日固定在某一天（例如每月1号）2.当前日期是2023-10-23（假设处理当日）3.借款完成日为2023-10-25revealed正常流动能力。结论：该用户本次借款周期基本是健康的，无异常波动迹象。，第 1 次重试...


情感分析中: 100%|██████████| 100/100 [01:38<00:00,  1.02it/s]


,label,review,llm_output
0,1,房间不大，但相当舒适．距离热闹的酒吧集中区也很近，方便．环境干净＼清净，适合睡大觉．服务中规...,1
1,1,酒店就是离市区太远。离新国际展会也不近。我的客人还算满意。,0
2,1,帮同事订过很多次，酒店位置很好，很方便。房间也不错。,1
3,0,"就值5元的浴盐瓶要赔40元,这不是主要的,最主要的是质量差的东西能伤人,我打开没用力就破,我...",0
4,0,帮朋友订的，反映不是太好，想必也是和城市规模有关系，宜昌的四星肯定是不能和大城市的四星相比。...,0


# 大模型分类效果评估

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df_multi = pd.read_csv('data_multithread.csv')
df_single = pd.read_csv('data_singlethread.csv')

def print_metrics(y_true, y_pred, prefix="结果"):
    acc = accuracy_score(y_true, y_pred)
    pre = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    print(f"{prefix}:")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Precision: {pre:.4f}")
    print(f"  Recall:    {rec:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print()

# df_multi
print_metrics(df_multi['label'], df_multi['llm_output'], prefix="多线程结果")

# df_single
print_metrics(df_single['label'], df_single['llm_output'], prefix="单线程结果")


多线程结果:
  Accuracy:  0.8400
  Precision: 0.9474
  Recall:    0.7200
  F1 Score:  0.8182

单线程结果:
  Accuracy:  0.8300
  Precision: 1.0000
  Recall:    0.6600
  F1 Score:  0.7952

